# Init OpenAI Client

In [11]:
from openai import AsyncOpenAI
import os

api_base_url = "http://localhost:8080" # api base url 
base_url = f"{api_base_url}/v1"
auth_key = 'sk-1234' # "sk-ragdondin-1234" # your api authentification key from your .env

client = AsyncOpenAI(api_key=auth_key, base_url=base_url)

In [12]:
await client.models.list() # listing existing openai models

AsyncPage[Model](data=[Model(id='ragondin-test', created=1747300932, object='model', owned_by='RAGondin'), Model(id='ragondin-all', created=0, object='model', owned_by='RAGondin')], object='list')

* chat settings

In [13]:
model = 'ragondin-all'
settings = {
    'model': model,
    'temperature': 0.7,
    'stream': False
}

# Testing `/chat/completions`

In [46]:
# init openai client

messages = [
    {
        "role": 'user',
        'content': 'Quelques fait sur la loi sapin 2'
    }
]

* **`Streaming mode`**

In [47]:
## 1. Streaming

stream = True
settings.update({"stream": stream})
stream_response = client.chat.completions.create(messages=messages, **settings)

async for chunk in await stream_response:
    print(chunk)

ChatCompletionChunk(id='chat-a07a1eef753b48908780c17fa206e956', choices=[Choice(delta=ChoiceDelta(content='Vo', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1747302701, model='ragondin-all', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None, sources='[{"doc_id": "[doc_1]", "file_url": "http://localhost:8080/static/test.txt", "filename": "test.txt", "_id": 458044157728658637, "chunk_url": "http://localhost:8080/extract/458044157728658637", "page": 13}, {"doc_id": "[doc_2]", "file_url": "http://localhost:8080/static/test.txt", "filename": "test.txt", "_id": 458044157728658639, "chunk_url": "http://localhost:8080/extract/458044157728658639", "page": 15}, {"doc_id": "[doc_3]", "file_url": "http://localhost:8080/static/test.txt", "filename": "test.txt", "_id": 458044157728658640, "chunk_url": "http://localhost:8080/extract/458044157728658640", "page": 15}, {"doc_id": "[doc_4]"

* Extracting Sources

In [48]:
import json
json.loads(chunk.sources)

[{'doc_id': '[doc_1]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658637,
  'chunk_url': 'http://localhost:8080/extract/458044157728658637',
  'page': 13},
 {'doc_id': '[doc_2]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658639,
  'chunk_url': 'http://localhost:8080/extract/458044157728658639',
  'page': 15},
 {'doc_id': '[doc_3]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658640,
  'chunk_url': 'http://localhost:8080/extract/458044157728658640',
  'page': 15},
 {'doc_id': '[doc_4]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658642,
  'chunk_url': 'http://localhost:8080/extract/458044157728658642',
  'page': 16},
 {'doc_id': '[doc_5]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658641,
  'chun

* **`Non-Streaming mode`**

In [22]:
# 2. Non Streaming

stream = False
settings.update({"stream": stream})

res = await client.chat.completions.create(
    messages=messages,
    **settings
)

In [23]:
import pprint

pprint.pprint(res.choices[0].message.content)

('Voici un exemple de dérive du lobbying :\n'
 '\n'
 '**Le cas des pesticides et de la multinationale Monsanto**\n'
 '\n'
 'En 2017, des documents internes de Monsanto, une multinationale de '
 'biotechnologie, ont été révélés sous le nom de « Monsanto Papers ». Ces '
 'documents montrent comment Monsanto a utilisé des stratégies de lobbying '
 'agressives pour influencer les décisions sur le glyphosate, un pesticide '
 "largement utilisé dans l'agriculture.\n"
 '\n'
 'Les Monsanto Papers révèlent que Monsanto a utilisé plusieurs techniques de '
 'lobbying pour influencer les décisions des autorités de régulation :\n'
 '\n'
 "*   **Ghostwriting** : Monsanto a payé des scientifiques pour qu'ils "
 'écrivent des articles sur les avantages du glyphosate, sans révéler leur '
 "lien avec l'entreprise.\n"
 '*   **Influence sur les décideurs** : Monsanto a utilisé des relations avec '
 'des décideurs politiques pour influencer leurs décisions, sans révéler son '
 'intérêt personnel.\n'
 '*   

# Testing `/completions`

* Testing the **`/completions`** endpoint with streaming
> It should fail cause the endpoint isn't compatible with streaming

In [36]:
# Testing /completions with streaming=True. It should fail

settings.update({"stream": True, })
response = await client.completions.create(prompt='La loi sapin 2, ', **settings)

print(response)

BadRequestError: Error code: 400 - {'detail': 'Streaming is not supported for this endpoint'}

* Testing the **`/completions`** endpoint without streaming

In [39]:
settings.update({"stream": False, 'max_tokens': 200})
response = await client.completions.create(prompt="La loi sapin 2, ne permet pas", **settings)

In [40]:
response.choices[0].text

" une transparence complète des relations entre décideurs publics et représentants d'intérêts. Les responsables publics ne sont pas soumis à aucune obligation de transparence dans leurs contacts avec les lobbies, alors que ce sont eux qui in fine arbitrent et prennent les décisions après les avoir entendus. Le répertoire des représentants d'intérêts ne fournit pas suffisamment de données utiles, et les obligations ne pèsent que sur les représentants d'intérêts.\n\nLa chronologie des régulations du lobbying dans les pays de l'OCDE montre que la régulation du lobbying a commencé relativement tôt aux États-Unis en 1940, mais que la plupart des pays de l'OCDE ont mis en place des régulations ou des codes de conduite plus tardivement, avec une concentration importante dans les années 2000.\n\nUn registre"

In [43]:
json.loads(response.sources)

[{'doc_id': '[doc_1]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658651,
  'chunk_url': 'http://localhost:8080/extract/458044157728658651',
  'page': 21},
 {'doc_id': '[doc_2]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658632,
  'chunk_url': 'http://localhost:8080/extract/458044157728658632',
  'page': 11},
 {'doc_id': '[doc_3]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658657,
  'chunk_url': 'http://localhost:8080/extract/458044157728658657',
  'page': 24},
 {'doc_id': '[doc_4]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658615,
  'chunk_url': 'http://localhost:8080/extract/458044157728658615',
  'page': 1},
 {'doc_id': '[doc_5]',
  'file_url': 'http://localhost:8080/static/test.txt',
  'filename': 'test.txt',
  '_id': 458044157728658638,
  'chunk